<h1 align="center">Dataset 2: Reporte de los registros al servicio 311 del gobierno NYC</h1>

In [2]:
%load_ext autoreload
%autoreload 2
# Gestion de rutas
import utils.paths as path
# Importacion de librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import os
import re
# Directorio de los datos crudos / raw data
raw_data_dir = path.make_dir_function(['datasets', 'raw'])
proc_data_dir = path.make_dir_function(['datasets', 'processed'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


* Reporte de los registros al servicio 311 del gobierno NYC

Descarga de datos de: https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [3]:
# Referencia: https://dongonchoi.github.io/RDataScience_Project/#materials_and_methods
import csv
import requests
import pandas as pd
import os

In [4]:
def download_data(url, file_name, start_date, end_date):
    # Ruta para guardar el archivo
    file_path = raw_data_dir(file_name)
    # Parámetros de paginación
    limit = 1000000  # Sin límite de resultados por página
    offset = 0    # Número de resultados a omitir antes de comenzar a obtener resultados
    results = []  # Lista para almacenar los resultados
    while True:
        # Construir la URL con los parámetros de paginación y el rango de fechas
        # query_url = f'{url}?$limit={limit}&$offset={offset}&$where=application_start_date >= "{start_date}" and application_start_date < "{end_date}"'
        query_url = f'{url}?$limit={limit}&$offset={offset}&$where=created_date >= "{start_date}" and created_date < "{end_date}"'
        # Hacer la solicitud a la API
        response = requests.get(query_url)
        # Verificar el código de estado de la respuesta
        if response.status_code == 200:
            # Leer los datos de la respuesta en un DataFrame
            df = pd.read_json(response.content)
            # Agregar los resultados al DataFrame
            results.append(df)
            # Verificar si hay más resultados
            if len(df) < limit:
                break  # Si no hay más resultados, salir del ciclo
            offset += limit  # Incrementar el offset para la próxima página
        else:
            print(f'Error al hacer la solicitud: {response.status_code}')
            break
    # Concatenar todos los DataFrames en uno solo
    df_final = pd.concat(results, ignore_index=True)
    try:
        # Guardar los datos en un archivo CSV
        df_final.to_parquet(file_path, index=False)
        print(f'Los datos han sido guardados en {file_path} exitosamente.')
    except Exception as e:
        print(f'Error al guardar los datos: {e}')

In [5]:
# Descarga del dataset 2
download_data(
    'https://data.cityofnewyork.us/resource/erm2-nwe9.json',
    'noise_pollution_ds2.parquet',
    '2020-01-01T00:00:00.000',
    '2023-01-01T00:00:00.000')

In [ ]:
# Lectura del dataset de reporte de los registros al servicio 311 del gobierno NYC
noise = pd.read_parquet(raw_data_dir('noise_pollution_ds2.parquet'))
noise.head()

Otras bases de datos a seguir explorando:
* https://data.cityofnewyork.us/Social-Services/311-Noise-Complaints/p5f6-bkga

* [Average (Leq) Noise Levels in dBA, by Transit Type and Measurement Location: New York City, June and July 2007](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2707461/table/tbl1/?report=objectonly)

* htps://www.nytimes.com/interactive/2020/05/22/upshot/coronavirus-quiet-city-noise.html

* https://data.cityofchicago.org/resource/ydr8-5enu.json